In [57]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
from datetime import date, datetime

In [7]:
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
model = os.environ.get('OPENAI_MODEL')
client = OpenAI(api_key=api_key)

In [20]:
# Open the JSON file for reading
with open('data/nba.json', 'r', encoding="utf8") as file:
    # Load the JSON data into a dictionary
    data = json.load(file)

hot_posts = data['hotposts']
gpt_input = ""
for hot_post in hot_posts:
    title = hot_post['title']
    body = hot_post['selftext']
    comments_arr = hot_post['comments']['data']['children']
    comments = ""
    for comment in comments_arr:
        comment_text = comment['data']['body']
        comments += comment_text + ','
    formatted_hot_post = 'Post title: title' + '\n' + ' Post body: ' + body + '\n' + 'Post comments: ' + comments
    gpt_input += formatted_hot_post + '\n'

In [40]:
completion = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "assistant", 
     "content": 
     "You are a daily newsletter writer that summarizes the past day's hot posts on a subreddit, with a chill, informative, modern narrative tone."},
     {"role": "user", "content": f"Provide me a newsletter title and body for the following subreddit's hot posts and comments, without an intro and signoff. Format response by including \"Newsletter Title:\" and \"Newsletter Body:\" in your response.: {gpt_input}."}
  ],
  temperature=0.3,
  max_tokens=800
)

In [41]:
raw_text = completion.choices[0].message.content

In [42]:
raw_text

"Newsletter Title: Hoops Highlights Daily\n\nNewsletter Body:\nGiannis Antetokounmpo showcased his dominance against the Hawks, leading the Bucks to victory with an impressive stat line. Despite some fans questioning his MVP candidacy, Giannis continues to shine, proving his worth in the league. The Bucks' rollercoaster season hasn't dampened Giannis' performance, and his consistency is a testament to his skill.\n\nIn another game, the spotlight was on Westbrook as he received love from OKC fans. His contributions to the Thunder franchise were celebrated, contrasting with the reception Durant faced. Westbrook's legacy continues to grow, with fans eager to see him excel on the court.\n\nMeanwhile, discussions around signature stat lines and career milestones in the NBA sparked debates among fans. From LeBron's impressive stats to Giannis' consistent performance, the league's top players are making history with their achievements.\n\nOn a lighter note, Jeff Teague's humorous comparison o

In [43]:
title_start = raw_text.index("Newsletter Title:") + len("Newsletter Title:")
title_end = raw_text.index("Newsletter Body:")
title =  raw_text[title_start:title_end].strip()

body_start = raw_text.index("Newsletter Body:") + len("Newsletter Body:")
body = raw_text[body_start:].strip()

In [44]:
title

'Hoops Highlights Daily'

In [45]:
body

"Giannis Antetokounmpo showcased his dominance against the Hawks, leading the Bucks to victory with an impressive stat line. Despite some fans questioning his MVP candidacy, Giannis continues to shine, proving his worth in the league. The Bucks' rollercoaster season hasn't dampened Giannis' performance, and his consistency is a testament to his skill.\n\nIn another game, the spotlight was on Westbrook as he received love from OKC fans. His contributions to the Thunder franchise were celebrated, contrasting with the reception Durant faced. Westbrook's legacy continues to grow, with fans eager to see him excel on the court.\n\nMeanwhile, discussions around signature stat lines and career milestones in the NBA sparked debates among fans. From LeBron's impressive stats to Giannis' consistent performance, the league's top players are making history with their achievements.\n\nOn a lighter note, Jeff Teague's humorous comparison of group projects resonated with fans, showcasing his laid-back

In [48]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate("../reddit-newsletter-firebase-key.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [59]:
today = date.today()
firestore_today = datetime(today.year, today.month, today.day)

In [60]:
doc_ref = db.collection("daily_subreddit_summary").document(f"nba - {today}")
doc_ref.set({"date": firestore_today, "title": title, "body": body})

update_time {
  seconds: 1711930232
  nanos: 554743000
}